### **网络结构**


原本的模型网络结构`feature_vec (in_dim) → concat_mlp(256) → 6个策略头 + 价值头`

当前的结构：在`mlp`后面加入自注意力模块，将其拆开成`T=4`的虚拟`token`，每个维度`D=64`，通过两层`Transformer Encoder`。对`token`进行平均池化，最后汇聚回256维和原来的256维做残差融合+LayerNorm

加入自注意力机制后的网络结构：\
`feature_vec (in_dim) → concat_mlp(256) → [自注意力机制] → 6个策略头 + 价值头`


### **具体代码**

在`conf/conf.py`中加入自注意力机制的参数：
```python
USE_SELF_ATTENTION = True   # 开关
SA_TOKENS = 4               # 虚拟token数T
SA_DIM = 64                 # 每个token维度D，需满足 T * D == 256
SA_HEADS = 4                # Multi-Head
SA_LAYERS = 2               # Transformer Encoder 层数
SA_DROPOUT = 0.0            # 注意力/FFN dropout
```

模型初始化中加入自注意力机制结构：
```python
self.use_self_attn = bool(getattr(Config, "USE_SELF_ATTENTION", True))
sa_tokens  = int(getattr(Config, "SA_TOKENS", 4))   # T
sa_dim     = int(getattr(Config, "SA_DIM", 64))     # D
sa_heads   = int(getattr(Config, "SA_HEADS", 4))
sa_layers  = int(getattr(Config, "SA_LAYERS", 2))
sa_dropout = float(getattr(Config, "SA_DROPOUT", 0.0))

# 防止错误
assert sa_tokens * sa_dim == 256, f"Self-Attn shape mismatch: {sa_tokens} * {sa_dim} != 256"

# 将256维共享表征拆为T个token，每个D维
self.attn_token_proj = nn.Linear(256, sa_tokens * sa_dim)

# Transformer编码层
encoder_layer = nn.TransformerEncoderLayer(
    d_model=sa_dim,
    nhead=sa_heads,
    dim_feedforward=sa_dim * 4,
    dropout=sa_dropout,
    activation="gelu",
    batch_first=True,
)
self.attn_encoder = nn.TransformerEncoder(encoder_layer, num_layers=sa_layers)

# 将编码后的pooled表征映回256，并做LayerNorm残差融合
self.attn_out_proj = nn.Linear(sa_dim, 256)
self.attn_ln = nn.LayerNorm(256)
```

```python
if self.use_self_attn:
    B = fc_public_result.size(0)
    # 256 → T*D → [B, T, D]
    tokens = self.attn_token_proj(fc_public_result).view(B, -1, int(getattr(Config, "SA_DIM", 64)))
    tokens = self.attn_encoder(tokens)             # [B, T, D]
    pooled = tokens.mean(dim=1)                    # [B, D] 平均池化
    attn_feat = self.attn_out_proj(pooled)         # [B, 256]
    fc_public_result = self.attn_ln(fc_public_result + attn_feat)  # 残差+LN保稳
```


### **整体流程**


#### **重要核心结构与关键配置**

**模型类**：`Modek(nn.Module)`

1. `concat_mlp`特征共享表征

结构：两层mlp, [in_dim -> 256 -> 256]最后一层保留激活

输入维度`in_dim`来自`DimConfig.DIM_OF_FEATURE[0]`

2. 自注意力模块

3. 多头策略分支

6个分支与`Config.LABEL_SIZE_LIST`对齐，每个分支都是`MLP(256→256→label_dim_i)`，输出对应动作分量的`logits`

4. 价值分支

`MLP(256→256→1)`，输出状态价值`V(s)`，用于PPO的优势/价值损失

**前向流程**：(记 batch 大小为 B，配置默认 T=4, D=64, T*D=256)


1. 共享特征
```python
fc_public_result = self.concat_mlp(feature_vec)  # [B, 256]
```

2. 自注意力
```python
tokens   = self.attn_token_proj(fc_public_result)      # [B, 256]
tokens   = tokens.view(B, T, D)                        # [B, 4, 64]
tokens   = self.attn_encoder(tokens)                   # [B, 4, 64]
pooled   = tokens.mean(dim=1)                          # [B, 64]
attn_feat= self.attn_out_proj(pooled)                  # [B, 256]
y        = self.attn_ln(fc_public_result + attn_feat)  # [B, 256]
# 当不使用自注意力机制时，y=fc_public_result
```

3. 多头策略输出
```python
for each i in range(6):
    logits_i = self.label_mlp[f"hero_label{i}_mlp"](y)   # [B, label_dim_i]
```

4. 价值输出
```python
value = self.value_mlp(y)   # [B, 1]
```

5. 返回值

训练时：返回`[logits_0, …, logits_5, value]`
推理时：把6个logits拼接成大向量，并且附上`value, lstm_cell_output, lstm_hidden_output`

#### **参数设置**

`T=4, D=64, heads=4, L=2, dropout=0.0`，出现震荡时可以`L=1`或`dropout=0.0`



### **补充解释**

1. 目前网络结构未真正使用LSTM(以后需要加入)，由于是对战类的游戏环境，需要加入LSTM以获取敌方的信息(过去若干步的隐藏状态信息)

2. 当前策略输出是6个分支(heads)的logits列表，与`Config.LABEL_SIZE_LIST`一一对应，具体内容包括如下：\
    1. 动作类型(head0)12维
    2. 参数化分支(head1-4)16维
    3. 目标选择(head5)9维
注意会配合合法动作掩码进行屏蔽非法动作

3. 训练/评估的`forward`返回值

- 训练模式：
```python
[
  logits_head_0: [B, label_dim_0],
  logits_head_1: [B, label_dim_1],
  logits_head_2: [B, label_dim_2],
  logits_head_3: [B, label_dim_3],
  logits_head_4: [B, label_dim_4],
  logits_head_5: [B, label_dim_5],
  value:         [B, 1]
]
```

- 评估模式：
```python
[
  concat_logits: [B, sum(label_dim_i)],  # 把6个head的logits沿特征维拼起来
  value:         [B, 1],
  lstm_cell_out,                         # 占位(未真正完成设计)
  lstm_hidden_out
]
```

训练模式为多头返回，评估模式拼接返回。

### **后序工作**

1. 可以继续对网络结构进行优化，当前只加入了自注意力机制，简单的`token`划分，未针对具体的情况进行划分，同时可以考虑对`token`进行汇聚而不是直接使用平均池化

2. 对于LSTM，后序在加入其他的特征之后需要启用，循环神经网络的记忆能力对对战类游戏非常有必要
